In [1]:
from db.duckdb.duckdbhelper import DuckDBDatabaseHelper
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import KMeans
from pprint import pprint

In [2]:
db = DuckDBDatabaseHelper("meters.db")
db.connect()

Connected to DuckDB database: meters.db


In [3]:
sql_query ="""

SELECT SUM(energy_sum),LCLid
FROM  meters m 
WHERE 
date_part('year', day) = 2014
GROUP BY LCLid
"""

db.connect()
records = db.fetch_all(sql_query)
db.close_connection()

Connected to DuckDB database: meters.db
Fetched 5108 rows.
Connection closed.


In [4]:
energy_sum = pd.DataFrame(records,columns= ["energy_sum",
                                            "LCLid"]).fillna(0)

In [5]:
n_energy_sum = np.array(energy_sum["energy_sum"])

In [6]:
kmeans = KMeans(n_clusters=5, 
                random_state=0,
                  n_init="auto").fit(n_energy_sum.reshape(-1, 1))
labels = list(kmeans.labels_)

In [7]:
clusters = pd.DataFrame()

In [8]:
clusters["energy_sum"] = energy_sum["energy_sum"]

In [9]:
clusters["labels"] = labels

In [10]:
clusters

,energy_sum,labels
0,1580.867000,4
1,329.584000,0
2,170.392000,0
3,682.977001,2
4,713.205000,2
...,...,...
5103,218.444000,0
5104,737.409000,2
5105,250.314000,0
5106,404.023000,0


In [11]:
result = clusters.groupby('labels')['energy_sum'].agg(['median', 'count'])

In [12]:
result.sort_values(by="median",ascending=False)

,median,count
labels,,
3,3284.5660,53
4,1908.3070,229
1,1110.6895,724
2,650.7710,1719
0,308.4450,2383


In [13]:
energy_sum.head()

,energy_sum,LCLid
0,1580.867000,MAC000216
1,329.584000,MAC004541
2,170.392000,MAC004591
3,682.977001,MAC004597
4,713.205000,MAC000572


In [14]:
len(energy_sum),len(clusters)

(5108, 5108)

In [15]:
energy_sum["cluster_label"] = clusters["labels"]

In [16]:
energy_sum.head()

,energy_sum,LCLid,cluster_label
0,1580.867000,MAC000216,4
1,329.584000,MAC004541,0
2,170.392000,MAC004591,0
3,682.977001,MAC004597,2
4,713.205000,MAC000572,2


In [17]:
energy_sum[energy_sum["cluster_label"] == 2].head()

,energy_sum,LCLid,cluster_label
3,682.977001,MAC004597,2
4,713.205000,MAC000572,2
7,745.050000,MAC004554,2
11,790.101000,MAC002955,2
13,875.609999,MAC002990,2


In [18]:
energy_sum[energy_sum["cluster_label"] == 4].head()

,energy_sum,LCLid,cluster_label
0,1580.867000,MAC000216,4
5,1756.705000,MAC005070,4
22,1618.586002,MAC000689,4
43,1704.477997,MAC003276,4
106,1994.858000,MAC000780,4


In [19]:
sql_query = """
DROP TABLE IF EXISTS cluster_energy 
"""
db.connect()
records = db.execute_query(sql_query)
db.close_connection()

Connected to DuckDB database: meters.db
Query executed successfully.
Connection closed.


In [20]:
sql_query = """
CREATE TABLE IF NOT EXISTS cluster_energy AS
SELECT * FROM energy_sum
"""
db.connect()
db.register_df("energy_sum",energy_sum)
records = db.execute_query(sql_query)
db.close_connection()

Connected to DuckDB database: meters.db
Query executed successfully.
Connection closed.


In [21]:
sql_query = """
SELECT * FROM cluster_energy
"""
db.connect()
records = db.fetch_all(sql_query)
db.close_connection()

Connected to DuckDB database: meters.db
Fetched 5108 rows.
Connection closed.


In [22]:
records[:4]

[(1580.8669995, 'MAC000216', 4),
 (329.584, 'MAC004541', 0),
 (170.392, 'MAC004591', 0),
 (682.9770005000003, 'MAC004597', 2)]

In [24]:
db.connect()
sql_query ="""
SELECT table_name, table_type 
FROM information_schema.tables;
"""
records = db.fetch_all(sql_query)
pprint(records)

db.close_connection()

Connected to DuckDB database: meters.db
Fetched 5 rows.
[('cluster_energy', 'BASE TABLE'),
 ('info_household', 'VIEW'),
 ('meters', 'VIEW'),
 ('weather_daily_darksky', 'VIEW'),
 ('weather_daily_darksky_modified', 'VIEW')]
Connection closed.
